# Building heights from EA LiDAR data
Generate building heights from DSM and DEM LiDAR data products available from the Environment Agency and apply them to OS OpenMap building outlines.

Steps:
1. Subtract DEM from DSM
2. Use zonal stats to get the mean height of the building
3. Write out to a new shapefile 

Data sourced from:
* LiDAR data products 
  https://environment.data.gov.uk/ds/survey/
* Building outlines 
  https://www.ordnancesurvey.co.uk/business-and-government/products/os-open-map-local.html
* Basemap for final image
  http://www.openstreetmap.org/copyright

In [ ]:
from osgeo import gdal
import glob
import numpy
from rasterstats import zonal_stats
import rasterio
import fiona

This function subtracts the DEM values from the DSM values to get the difference.  The mean of the values that fall within a building outline are then applied to those outlines to give them a height value.

In [ ]:
def getheights(dtmfile, dsmfile, zoneshp, outshp):
    """
    Apply heights derived from elevation data to building outlines
    Parameters
    ----------
    dtmfile : string
        Path to dtm file
    dsmfile : string
        Path to dsm file
    zoneshp : string
        Path to shapefile to use for building outlines
    outshp : string
        Path to out shapefile
    """

    with rasterio.open(dtmfile) as dtm:
        dtmarray = dtm.read(1)
        dtmaffine = dtm.affine

    with rasterio.open(dsmfile) as dsm:
        dsmarray = dsm.read(1)
        # dsmaffine = dsm.affine

    #------------------------------------------------------
    # take the dtm heights from the dsm to get the height
    #------------------------------------------------------
    heightarray = dsmarray - dtmarray

    #--------------
    # zonal stats
    #--------------
    stats = zonal_stats(zoneshp,
                        heightarray,
                        affine=dtmaffine,
                        stats="mean",
                        geojson_out=True)

    # get the input schema to use for output
    with fiona.open(zoneshp) as input:
        output_schema = input.schema.copy()

    # add stats fields to the schema
    output_schema['properties']['mean'] = 'float'

    # write out to shapefile
    with fiona.open(heightshp, 
                    'w', 
                    'ESRI Shapefile', 
                    output_schema, 
                    crs=input.crs) as output:
        for i, feat in enumerate(stats): 
            fproperties = stats[i]['properties']
            fgeometry = stats[i]['geometry']

            output.write({'properties': fproperties,'geometry': fgeometry})

The data that is downloaded from the EA website comes in a series of smaller tiles.  These are joined into a virtual raster tile (VRT) using GDAL.  The VRT is then fed into the function to derive building heights.

In [ ]:
dtmvrt  = "data/LIDAR-DTM-1M-WL.vrt"
dsmvrt  = "data/LIDAR-DSM-1M-WL.vrt"
zoneshp = "data/Building_WL.shp"
heightshp = "data/wl_heights.shp"

#----------------------------------
# BUILD VRTS FROM THE LIDAR TILES
#----------------------------------

# DTM
gdal.BuildVRT(dtmvrt, glob.glob("data/dtm/LIDAR-DTM-1M-*/*.asc"))

# DSM
gdal.BuildVRT(dsmvrt, glob.glob("data/dsm/LIDAR-DSM-1M-*/*.asc"))

#----------------------------------
# GET HEIGHTS
#----------------------------------
getheights(dtmvrt, dsmvrt, zoneshp, heightshp)

The resulting shapefile was visualised in ArcGIS Pro to produce the image below where the buildings are overlaid on an Open Street Map baselayer.

<img src="images/building_heights.png">